In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
import time

In [2]:
options = webdriver.ChromeOptions()
# Using SOCKS Proxy (Optional)
options.add_argument("--proxy-server=socks5://127.0.0.1:8081")
browser = webdriver.Chrome('f:/utils/chromedriver/chromedriver.exe', options=options)

In [3]:
browser.get("https://apps.wv.gov/SOS/BusinessEntitySearch/")

In [6]:
entity = browser.find_element_by_id("phMain_txtOrganizationName")
entity.clear()
entity.send_keys("son")
search = browser.find_element_by_id("phMain_btnSearch")
search.click()

In [24]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

df = None
page = 1

In [ ]:
while True:
    try:
        print('page = %d' % page)
        p = page % 10
        if p == 0:
            p = 10
        if page != 1 and p == 1:
            elem = browser.find_element_by_id("phMain_gvSearchResults_lbPageNext")
            elem.click()
            time.sleep(3)
        elem = browser.find_element_by_id("phMain_gvSearchResults_lbPage%d" % p)
        elem.click()
    except Exception as e:
        print(e)
        break
    time.sleep(3)
    dfs = pd.read_html(browser.page_source, attrs={'id': 'phMain_gvSearchResults'})
    if df is None:
        df = dfs[0][:-1]
    else:
        df = pd.concat([df, dfs[0][:-1]])
    page += 1
    if page > 100:
        break

In [33]:
df

,Name,Type,City,Status,Details
0,D. HAYES & SON CONTRACTING,NaN,NaN,Active,Details
1,GIBSONS CONSULTING,NaN,NaN,Active,Details
2,KIMBERLEY S PIERSON,NaN,NaN,Active,Details
3,"""MODERN MASONRY CONCEPTS"" LLC",Limited Liability Company,REEDY,Not Active,Details
4,"""SAVAGE"" STEVENSON INC.",Corporation,HUNTINGTON,Not Active,Details
...,...,...,...,...,...
5,"BERNARD G. SAMPSON COMPANY, INC.",Corporation,FAIRMONT,Not Active,Details
6,BERNARD JOHNSON CORPORATION,Corporation,AUSTIN,Not Active,Details
7,BERNICE WILLIAMSON,NaN,NaN,Active,Details
8,BERRY WILSON DRILLING COMPANY,Corporation,HUNTINGTON,Not Active,Details


In [34]:
df.reset_index(drop=True).to_csv('son-wv-1k.csv', index=False)

In [35]:
browser.quit()